## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

ModuleNotFoundError: No module named 'config'

In [ ]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 


ValueError: could not convert string to float: 

In [57]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mana,GF,5.66,-53.78,78.76,88,8,5.14,clear sky
1,1,North Bend,US,43.41,-124.22,81.00,72,1,25.28,clear sky
3,3,Olinda,BR,-8.01,-34.86,78.80,78,75,10.29,broken clouds
4,4,Upata,VE,8.01,-62.40,82.00,76,32,1.99,scattered clouds
6,6,Zhuhai,CN,22.28,113.57,86.00,84,20,3.36,few clouds
16,16,Atuona,PF,-9.80,-139.03,77.63,80,50,19.77,light rain
18,18,Georgetown,MY,5.41,100.34,80.60,83,20,3.33,few clouds
21,21,Hambantota,LK,6.12,81.12,77.00,94,100,4.70,heavy intensity rain
27,27,Itarema,BR,-2.92,-39.92,78.12,78,0,16.67,clear sky
28,28,Kanth,IN,29.07,78.63,77.05,85,93,4.74,overcast clouds


In [58]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [59]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# There are no empty rows because there are values in all the cells
preferred_cities_df.notnull().sum()

City_ID                250
City                   250
Country                250
Lat                    250
Lng                    250
Max Temp               250
Humidity               250
Cloudiness             250
Wind Speed             250
Current Description    250
dtype: int64

In [60]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Mana,GF,78.76,clear sky,5.66,-53.78,
1,North Bend,US,81.00,clear sky,43.41,-124.22,
3,Olinda,BR,78.80,broken clouds,-8.01,-34.86,
4,Upata,VE,82.00,scattered clouds,8.01,-62.40,
6,Zhuhai,CN,86.00,few clouds,22.28,113.57,
16,Atuona,PF,77.63,light rain,-9.80,-139.03,
18,Georgetown,MY,80.60,few clouds,5.41,100.34,
21,Hambantota,LK,77.00,heavy intensity rain,6.12,81.12,
27,Itarema,BR,78.12,clear sky,-2.92,-39.92,
28,Kanth,IN,77.05,overcast clouds,29.07,78.63,


In [61]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")   
    


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [64]:
# Check contents of hotel dataframe - there 250 records.
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Mana,GF,78.76,clear sky,5.66,-53.78,LE SAMANA
1,North Bend,US,81.00,clear sky,43.41,-124.22,Red Lion Hotel Coos Bay
3,Olinda,BR,78.80,broken clouds,-8.01,-34.86,Hotel 5 Sóis
4,Upata,VE,82.00,scattered clouds,8.01,-62.40,Posada Villa Nela
6,Zhuhai,CN,86.00,few clouds,22.28,113.57,珠海粤财假日酒店
...,...,...,...,...,...,...,...
681,Ruston,US,86.00,clear sky,32.52,-92.64,"Country Inn & Suites by Radisson, Ruston, LA"
685,La Asuncion,VE,84.20,clear sky,11.03,-63.86,Colinas de Matasiete
687,Ocos,GT,82.40,broken clouds,14.51,-92.19,La Bajadita
689,Port Arthur,CN,78.80,clear sky,38.80,121.27,Friendship Hotel


In [65]:
# 7. Drop the rows where there is no Hotel Name - there are now 238 complete records.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Mana,GF,78.76,clear sky,5.66,-53.78,LE SAMANA
1,North Bend,US,81.00,clear sky,43.41,-124.22,Red Lion Hotel Coos Bay
3,Olinda,BR,78.80,broken clouds,-8.01,-34.86,Hotel 5 Sóis
4,Upata,VE,82.00,scattered clouds,8.01,-62.40,Posada Villa Nela
6,Zhuhai,CN,86.00,few clouds,22.28,113.57,珠海粤财假日酒店
...,...,...,...,...,...,...,...
681,Ruston,US,86.00,clear sky,32.52,-92.64,"Country Inn & Suites by Radisson, Ruston, LA"
685,La Asuncion,VE,84.20,clear sky,11.03,-63.86,Colinas de Matasiete
687,Ocos,GT,82.40,broken clouds,14.51,-92.19,La Bajadita
689,Port Arthur,CN,78.80,clear sky,38.80,121.27,Friendship Hotel


In [66]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [68]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [1]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

NameError: name 'clean_hotel_df' is not defined